In [ ]:
#clear history
%reset -f

import numpy as np
np.random.seed(10)
import math

from enum import Enum
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
import random
import pandas as pd
#necessary for removing dublicates from a list of lists
import itertools


from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.batchrunner import BatchRunner


#import local ipnyb files
%run ./model.ipynb
%run ./agent.ipynb
%run ./model_output_functions.ipynb

In [ ]:
time_step_to_year = 52

In [ ]:
#MODEL1 

#time steps=2 weeks
#avg life= 60 years=~ 1500 time steps
#infection time=3months=~ 6 time steps
#time horizon= 5 years=~ 130 time steps
#life long immunity

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

strain_list = []
loci1 =[0,1,2]

for i in loci1:
    for j in loci1:
        strain_list.append([i,j])
        

n_pop=250
#start with single infection only (no co infection at the beginning)
n_infected_list=[5]*(len(strain_list))
avg_contacted_agents=10
prob_of_transmission_list=[0.05]*(len(strain_list))

no_loci_AT=1
no_loci_MT=1
#print(prob_of_transmission_list)
avg_recovery_time=4
n_iterations=10
immunity_duration=30
#no limit for max no infections
max_no_infections=len(strain_list)
birthdeath=True
timeinterval_diversity=100
migration_rate=0.01
carriage_percentage=0.9
no_runs=1
timediff=52
seed = 1
intervention_time = -1
for j in range(no_runs): 
    random.seed(0)
    mymodel = BaseModel(agent_type = HumanBase, n_pop=n_pop,n_infected_list=n_infected_list, strain_list=strain_list,
                   avg_contacted_agents=avg_contacted_agents, 
                   prob_of_transmission_list=prob_of_transmission_list,
                   avg_recovery_time=avg_recovery_time,
                   n_iterations=n_iterations,
                    agentDataTime=900,
                  no_loci_AT=no_loci_AT,no_loci_MT=no_loci_MT,
                   max_no_infections=max_no_infections,
                   immunity_duration=immunity_duration,timeinterval_diversity=timeinterval_diversity,
                  migration_rate=migration_rate ,carriage_percentage=carriage_percentage,
                  birthdeath=birthdeath,seed = seed,intervention_time = intervention_time,
                   #is_homogeneous=False
                  )
   
    variables = mymodel.datacollector.get_model_vars_dataframe()
    #total=sum([1 for t in variables.Infected[n_iterations] if t != 0])
    sum1=sum([t for t in variables.Infected[n_iterations] if t != 0])
    if (sum1 != 0):
        varInf=[i/sum1 for i in variables.Infected[n_iterations]]
        total=[[strain_list[i] for i,t in enumerate(varInf) if t >= 0.05]]
        nototal=sum([1 for t in varInf if t >= 0.05])
        percstrains= [[round(t,2) for t in varInf if t >= 0.05]]
    else: 
        total=[[]]
        percstrains=[[]]
        nototal=0
    #index of non zero &>0.5 left strains
    d1 = {'Left Pop': variables.TotalPop[n_iterations],
          'Left No strains': nototal,
          'Left strains':total,
          'Perc of Strains':percstrains,
          'Left Total Infected':variables.TotalInfected[n_iterations]}   
        #perc of infected of strain over all infections
    if (j==0):
        v=variables.copy()
        v['No']=[j]*v.shape[0]   #row = v.shape[0]
        df1=pd.DataFrame(data=d1, index=[0])
        plt.xlabel("Time")
        plt.ylabel("People")
        plt.title("Infections")
        ax = plt.subplot(111)
        for i in range(len(strain_list)):
            ax.plot(list(range(0, n_iterations+1)),
                     [row[i] for row in variables.Infected],
                     label='%s'%strain_list[i])
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(strain_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("Reff Plot")
        ax = plt.subplot(111)
        label_list=[]
        for i in range(len(strain_list)):
            ax.plot(list(range(0, n_iterations+1)),
                     [row[i] for row in variables.Reff],
                     label='Reff_%s'%strain_list[i])
            label_list.append('Reff_%s'%strain_list[i])
        
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogen, label="ReffPathogen")
        label_list.append("ReffPathogen")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("ReffWeekly Plot")
        ax = plt.subplot(111)
        label_list=[]
        for i in range(len(strain_list)):
            ax.plot(list(range(0, n_iterations+1)),
                     [row[i] for row in variables.ReffWeekly],
                     label='Reff_%s'%strain_list[i])
            label_list.append('Reff_%s'%strain_list[i])
        
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogenWeekly, label="ReffWeeklyPathogen")
        label_list.append("ReffWeeklyPathogen")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("ReffAT Plot")
        ax = plt.subplot(111)
        label_list=[]
        for i in range(3):
            ax.plot(list(range(0, n_iterations+1)),
                     [row[i] for row in variables.ReffAT],
                     label='Reff_%s'%i)
            label_list.append('Reff_%s'%i)
        
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogen, label="ReffPathogen")
        label_list.append("ReffPathogen")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("ReffATWeekly Plot")
        ax = plt.subplot(111)
        label_list=[]
        for i in range(3):
            ax.plot(list(range(0, n_iterations+1)),
                     [row[i] for row in variables.ReffATWeekly],
                     label='Reff_%s'%i)
            label_list.append('Reff_%s'%i)
        
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogenWeekly, label="ReffPathogenWeekly")
        label_list.append("ReffPathogenWeekly")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("Reff Pathogen")
        ax = plt.subplot(111)
        label_list=[]
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogen, label="ReffPathogen")
        label_list.append("ReffPathogen")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("Reff Pathogen Weekly")
        ax = plt.subplot(111)
        label_list=[]
        ax.plot(list(range(0, n_iterations+1)),
                     v.ReffPathogenWeekly, label="ReffPathogenWeekly")
        label_list.append("ReffPathogenWeekly")
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        # Put a legend to the right of the current axis
        ax.legend(label_list,loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
        plt.xlabel("Time")
        plt.title("CirculatingNoStrains")
        ax = plt.subplot(111)
        ax.plot(list(range(0, n_iterations+1)),
                     v.CirculatingNoStrains)
        plt.show()
        
        plt.xlabel("Time")
        plt.title("CirculatingNoATs")
        ax = plt.subplot(111)
        ax.plot(list(range(0, n_iterations+1)),
                     v.CirculatingNoATs)
        plt.show()
 
    else:
        v1=variables.copy()
        v1['No']=[j]*v1.shape[0]   #row = v.shape[0]
        v=v.append(v1,ignore_index = True)
        df2 = pd.DataFrame(data=d1, index=[0])
        df1=df1.append(df2, ignore_index = True) 
        
#display(v)
